In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
sys.path.append("..") # Adds higher directory to python modules path.
import branchingdnn as branching

In [2]:
dataset = branching.dataset.prepare.dataset(tf.keras.datasets.cifar10.load_data(),32,5000,22500,(227,227), include_targets=True, categorical=True)
train_ds, test_ds, validation_ds = dataset

augment Dataset
targetsis : True
adding targets to inputs
trainSize 45000
testSize 10000


In [37]:
inputs = keras.Input(shape=(227,227,3))
targets = keras.Input(shape=(10,))
x = keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3))(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
x = keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
x = keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(4096, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)

# ### first branch
# branchLayer = keras.layers.Flatten(name=tf.compat.v1.get_default_graph().unique_name("branch_flatten"))(x)
# branchLayer = keras.layers.Dense(124, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("branch124"))(branchLayer)
# branchLayer = keras.layers.Dense(64, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("branch64"))(branchLayer)
# branchLayer = keras.layers.Dense(10, name=tf.compat.v1.get_default_graph().unique_name("branch_output"))(branchLayer)

x = keras.layers.Dense(4096, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)

# ### second Branch
# branchLayer2 = keras.layers.Flatten(name=tf.compat.v1.get_default_graph().unique_name("branch_flatten"))(x)
# branchLayer2 = keras.layers.Dense(10, name=tf.compat.v1.get_default_graph().unique_name("branch_output"))(branchLayer2)

x = keras.layers.Dense(10, activation='softmax')(x)

# model = keras.Model(inputs=inputs, outputs=[x,branchLayer,branchLayer2], name="alexnet")
model = keras.Model(inputs=(inputs, targets), outputs=[x], name="alexnet")

checkpoint = keras.callbacks.ModelCheckpoint("models/alexNetv6_test.hdf5", monitor='val_loss',verbose=1,save_best_only=True, mode='auto',period=1)
model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001,momentum=0.9), metrics=['accuracy'])
model.summary()


model.fit(train_ds,
      epochs=10,
      validation_data=validation_ds,
      validation_freq=1,
      callbacks=[checkpoint])


Model: "alexnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 55, 55, 96)   34944       input_11[0][0]                   
__________________________________________________________________________________________________
batch_normalization_25 (BatchNo (None, 55, 55, 96)   384         conv2d_25[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_15 (MaxPooling2D) (None, 27, 27, 96)   0           batch_normalization_25[0][0]     
____________________________________________________________________________________________

In [35]:
dataset_teacher = branching.dataset.prepare.dataset(tf.keras.datasets.cifar10.load_data(),32,5000,22500,(227,227), include_targets=False,categorical=True)
train_ds_teacher, test_ds_teacher, validation_ds_teacher = dataset_teacher
model_teacher = tf.keras.models.load_model("models/alexNetv6_logits_teacher.hdf5")


augment Dataset
targetsis : False
trainSize 45000
testSize 10000


In [36]:
model_teacher.evaluate(test_ds_teacher)

312/312 [==============================] - 5s 14ms/step - loss: 0.6905 - accuracy: 0.7940


[0.6905297040939331, 0.7939703464508057]

In [10]:
 outputs =[]
inputs = keras.Input(shape=(28,28,))
x = layers.Flatten(input_shape=(28,28))(inputs)
x = layers.Dense(512, activation="relu")(x)
#exit 1 The main branch exit is refered to as "exit 1" or "main exit" to avoid confusion when adding addtional exits
output1 = layers.Dense(10, name="output1")(x)
softmax = layers.Softmax()(output1)

outputs.append(softmax)
print(len(outputs))
model_student = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model_normal")
model_student.summary()
#visualize_model(model,"mnist_normal")
print(len(model_student.outputs))


1
Model: "mnist_model_normal"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               401920    
_________________________________________________________________
output1 (Dense)              (None, 10)                5130      
_________________________________________________________________
softmax_3 (Softmax)          (None, 10)                0         
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
1


In [11]:
model_student.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model_student.fit(x_train, y_train, batch_size=batch_size, epochs=5, validation_split=0.1)

Epoch 1/5
422/422 [==============================] - 2s 3ms/step - loss: 0.4960 - accuracy: 0.8634 - val_loss: 0.1262 - val_accuracy: 0.9672
Epoch 2/5
422/422 [==============================] - 1s 3ms/step - loss: 0.1231 - accuracy: 0.9649 - val_loss: 0.0897 - val_accuracy: 0.9748
Epoch 3/5
422/422 [==============================] - 1s 3ms/step - loss: 0.0786 - accuracy: 0.9774 - val_loss: 0.0823 - val_accuracy: 0.9760
Epoch 4/5
422/422 [==============================] - 1s 3ms/step - loss: 0.0539 - accuracy: 0.9854 - val_loss: 0.0710 - val_accuracy: 0.9812
Epoch 5/5
422/422 [==============================] - 1s 3ms/step - loss: 0.0368 - accuracy: 0.9897 - val_loss: 0.0713 - val_accuracy: 0.9808


In [38]:
model_student.evaluate(test_ds_teacher)

ValueError: in user code:

    c:\users\sanity\appdata\local\conda\conda\envs\py36\lib\site-packages\tensorflow\python\keras\engine\training.py:1233 test_function  *
        return step_function(self, iterator)
    c:\users\sanity\appdata\local\conda\conda\envs\py36\lib\site-packages\tensorflow\python\keras\engine\training.py:1224 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\sanity\appdata\local\conda\conda\envs\py36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\sanity\appdata\local\conda\conda\envs\py36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\sanity\appdata\local\conda\conda\envs\py36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
       